In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import os
import io
import gzip


1. Lectura de los archivos en raw, y me creo los archivos parquet que voy a utilizar

In [7]:
train = pd.read_csv('../../data/raw/train.csv')
test_nolabel = pd.read_csv('../../data/raw/test_nolabel.csv')
sample_submission= pd.read_csv('../../data/raw/sample_submission.csv')

train.to_parquet('../../data/df_train.parquet')
test_nolabel.to_parquet('../../data/df_test_nolabel.parquet')
sample_submission.to_parquet('../../data/df_sample_submission.parquet')

#df_train = pd.read_parquet('../data/agregado_diario_2019-2021.parquet')

In [8]:
df_train = pd.read_parquet('../../data/df_train.parquet')
df_test_nolabel = pd.read_parquet('../../data/df_test_nolabel.parquet')
df_sample_submission = pd.read_parquet('../../data/df_sample_submission.parquet')

In [134]:
df_train.head()

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,1-Nov-05,2006,2,...,0,2,0,1,N,N,31-Dec-05,"$8,000.00",$0.00,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,6-Jun-05,2005,2,...,1,2,1,1,N,N,31-Jul-05,"$166,000.00",$0.00,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,18-Mar-03,2003,2,...,4,2,1,2,Y,N,31-Mar-03,"$25,000.00",$0.00,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,28-Jun-95,1995,2,...,0,0,1,0,N,N,31-Jan-96,"$220,100.00",$0.00,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,16-Apr-09,2009,0,...,0,0,0,1,N,N,31-May-09,"$25,000.00",$0.00,0


In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22835 entries, 0 to 22834
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 22835 non-null  object 
 1   LoanNr_ChkDgt      22835 non-null  int64  
 2   Name               22834 non-null  object 
 3   City               22834 non-null  object 
 4   State              22835 non-null  object 
 5   Bank               22813 non-null  object 
 6   BankState          22813 non-null  object 
 7   ApprovalDate       22835 non-null  object 
 8   ApprovalFY         22835 non-null  int64  
 9   NoEmp              22835 non-null  int64  
 10  NewExist           22833 non-null  float64
 11  CreateJob          22835 non-null  int64  
 12  RetainedJob        22835 non-null  int64  
 13  FranchiseCode      22835 non-null  int64  
 14  UrbanRural         22835 non-null  int64  
 15  RevLineCr          22744 non-null  object 
 16  LowDoc             227

In [18]:
df_train.isnull().sum()

id                    0
LoanNr_ChkDgt         0
Name                  1
City                  1
State                 0
Bank                 22
BankState            22
ApprovalDate          0
ApprovalFY            0
NoEmp                 0
NewExist              2
CreateJob             0
RetainedJob           0
FranchiseCode         0
UrbanRural            0
RevLineCr            91
LowDoc               43
DisbursementDate     61
DisbursementGross     0
BalanceGross          0
Accept                0
dtype: int64

2. Cambiar los valores correspondientes.

La gran mayoría de los valores son de tipo objet, int64 y float64, tienen que ser acorde a sus valores.

Primero cambio los valores de las fechas, que son Approval Date y DisbursementDate que están puesto con letra escrita en vez de en el formato de numeros.

* *DisbursementDate* => transformar la fech
* *Name* => String, Si no sabemos lo ponemos Unknown
* *City* => String, Si no sabemos lo ponemos Unknown
* *State* => Son todosde tipo OH
* *Bank* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *BankState* => Si quitamos los None es de tipo String, Si no sabemos lo ponemos Unknown
* *NewExist* => 2 / 1 / 0, int, y poner la media donde sea nulo; 

    1 = Existing business, 2 = New business; 

    Si no lo sabemos ponemos vieja empresa que es la mayoría, es decir, que ya existía la empresa, 0. 

    Por lo que 0 = Existing y 1 = New
* *RevLineCr* => array(['N', 'Y', '0', 'T', None, '`'], dtype=object), 

    Hay que limpiarlo, sabiendo que tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *LowDoc* => array(['N', 'Y', None, 'A', '0', 'S', 'R', 'C'], dtype=object)

    Tiene que ser Y = Yes, N = No, para el que no sepamos ponemos la mayoría; después ponemos True => 1 y False => 0
* *DisbursementGross* => Quitar el dolar y hacerlo double
* *BalanceGross* => Quitar el dolar y hacerlo double y donde sea nulo

In [128]:
df_train2 = df_train.copy()
df_train2['ApprovalDate'] = pd.to_datetime(df_train2['ApprovalDate'])
df_train2['DisbursementDate'] = pd.to_datetime(df_train2['DisbursementDate'])

df_train2['Name'].fillna('Unknown', inplace=True) 
df_train2['City'].fillna('Unknown', inplace=True) 
df_train2['Bank'].fillna('Unknown', inplace=True) 
df_train2['BankState'].fillna('Unknown', inplace=True) 
#Lo convertimos en Int y despues los Nan en numero, Nueva => True, 1; Si ya existe la ponemos como 0
df_train2['NewExist'] = df_train2['NewExist'].apply(
    lambda x: 0 if str(x).strip().replace('.0','') == '1'
              else (1 if str(x).strip().replace('.0','') == '2'
                    else np.nan))
df_train2['NewExist'].fillna(df_train2['NewExist'].mode()[0], inplace=True)
df_train2['NewExist'] = df_train2['NewExist'].astype(int)
######################Limpieza RevLineCr#
df_train2['RevLineCr'] = df_train2['RevLineCr'].apply(lambda x: 1 if str(x).strip().upper() in ['Y','T'] 
                                                        else (0 if str(x).strip().upper() in ['N','0'] 
                                                              else np.nan))
df_train2['RevLineCr'].fillna(df_train2['RevLineCr'].mode()[0], inplace=True)
df_train2['RevLineCr'] = df_train2['RevLineCr'].astype(int)
######################Limpieza LowDoc#
df_train2['LowDoc'] = df_train2['LowDoc'].apply(lambda x: 1 if str(x).strip().upper() in ['Y'] 
                                                  else (0 if str(x).strip().upper() in ['N','0'] 
                                                        else np.nan))
df_train2['LowDoc'].fillna(df_train2['LowDoc'].mode()[0], inplace=True)
df_train2['LowDoc'] = df_train2['LowDoc'].astype(int)
######################
#ordenar por fecha
df_train2.sort_values('DisbursementDate', ascending=True, inplace=True)
df_train2['DisbursementDate'].fillna(method='ffill', inplace=True) #El valor anterior no nulo despues de ordenarlo por fecha

#Limpiamos los dólares y lo ponemos en tipo float
df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
df_train2

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:5: SyntaxWarning: invalid escape sequence '\$'
C:\Users\ANGEL\AppData\Local\Temp\ipykernel_16648\866823969.py:4: SyntaxWarning: invalid escape sequence '\$'
  df_train2['DisbursementGross'] = df_train2['DisbursementGross'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\ANGEL\AppData\Local\Temp\ipykernel_16648\866823969.py:5: SyntaxWarning: invalid escape sequence '\$'
  df_train2['BalanceGross'] = df_train2['BalanceGross'].replace('[\$,]', '', regex=True).astype(float)
C:\Users\ANGEL\AppData\Local\Temp\ipykernel_16648\866823969.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_train2['ApprovalDate'] = pd.to_datetime(df_train2['ApprovalDate'])
C:\Users\ANGEL\AppData\Local\Temp\ipykerne

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
17317,53bd927f37d,7134351004,"HORTCO OF OHIO,INC.",NEW CARLISLE,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,1975-09-12,1976,6,...,0,0,0,0,0,0,1975-10-17,125000.0,0.0,0
1175,08b99c7464f,7206221000,JERRYS HANDY TOOL ETAL,DAYTON,OH,JPMORGAN CHASE BANK NATL ASSOC,OH,1976-05-26,1976,2,...,0,0,0,0,0,0,1976-08-05,30000.0,0.0,1
2961,05f518096ee,8175271005,APOLLO BEDDING INC,CLEVELAND,OH,KEYBANK NATIONAL ASSOCIATION,OH,1976-10-22,1977,9,...,0,0,0,0,0,0,1976-12-15,59000.0,0.0,0
4697,1899e68246d,8176641000,"LOX CONSTRUCTION, INC.",WOOSTER,OH,JPMORGAN CHASE BANK NATL ASSOC,OH,1977-03-25,1977,10,...,0,0,0,0,0,0,1977-06-21,400000.0,0.0,0
10696,b1d476889da,8945771010,DENTAL CONSUMABLES OF AMER INC,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,1978-01-13,1978,20,...,0,0,0,0,0,0,1978-01-31,800000.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21244,b5ca1479066,5652894002,INDEPENDENCE REHAB INC.,INDEPENDENCE,OH,"FIRSTMERIT BANK, N.A.",OH,2002-08-28,2002,2,...,4,2,1,1,1,0,2074-03-11,30000.0,0.0,1
21987,33c1e101cf1,1902125007,"Cross Country Distributing, In",Huber Heights,OH,FIFTH THIRD BANK,OH,2006-06-30,2006,20,...,22,22,0,1,1,0,2074-03-11,528257.0,0.0,1
22351,9f9d0b78449,5071024001,ELECTRO DYNAMICS INC,EASTLAKE,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2001-11-28,2002,8,...,4,8,1,1,1,0,2074-03-11,100000.0,0.0,1
22423,c95145b3ac9,5275344005,SPHERE LLC,LEWIS CENTER,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2002-03-14,2002,2,...,2,2,1,1,1,0,2074-03-11,35000.0,0.0,1


In [129]:
df_train2.isna().sum()

id                   0
LoanNr_ChkDgt        0
Name                 0
City                 0
State                0
Bank                 0
BankState            0
ApprovalDate         0
ApprovalFY           0
NoEmp                0
NewExist             0
CreateJob            0
RetainedJob          0
FranchiseCode        0
UrbanRural           0
RevLineCr            0
LowDoc               0
DisbursementDate     0
DisbursementGross    0
BalanceGross         0
Accept               0
dtype: int64

In [132]:
df_train2.BalanceGross.value_counts(dropna=False)

BalanceGross
0.0         22833
996262.0        1
9111.0          1
Name: count, dtype: int64